In [ ]:
import os
import pandas as pd
import cv2
import numpy as np
# Define the paths to the image and label files
IMG_PATH = "E:/mtech/secondsem/CV_Lab/Project/Hands" #whole images
LABEL_PATH = "E:/mtech/secondsem/CV_Lab/Project/HandInfo.csv" #information of images
# Load the label data from the CSV file
labels_df = pd.read_csv(LABEL_PATH)
labels_dict = {}

# Iterate over the rows of the CSV file and extract the labels for each image
for index, row in labels_df.iterrows():
    filename = row['imageName']
    label = row['aspectOfHand']
    labels_dict[filename] = label

# Load the images and labels into memory
images = []
labels = []
for filename in os.listdir(IMG_PATH):
    img = cv2.imread(os.path.join(IMG_PATH, filename))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    images.append(img)
    labels.append(labels_dict[filename])

# Convert the data to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Print the shape of the data arrays
print("Images shape:", images.shape)
print("Labels shape:", labels.shape)


# LSTM

In [1]:
import os
import pandas as pd
import cv2
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

from PIL import Image
import os
import numpy as np

# Define the paths to the image and label files
IMG_PATH = "E:/mtech/secondsem/CV_Lab/Project/Hands/Hands" #whole images
LABELS_PATH = "E:/mtech/secondsem/CV_Lab/Project/HandInfo.csv"

# Load the labels
labels_df = pd.read_csv(LABELS_PATH)

# Define the patch size
PATCH_SIZE = 32

# Define a list to store the image patches
image_patches = []

# Load and preprocess each image
for i, row in labels_df.iterrows():
    img_path = os.path.join(IMG_PATH, row["imageName"])
    img = Image.open(img_path)
    img = img.resize((224, 224)) # ResNet50 model takes input size of (224,224)
    
    # Divide the image into patches
    for j in range(0, img.size[0], PATCH_SIZE):
        for k in range(0, img.size[1], PATCH_SIZE):
            patch = img.crop((j, k, j+PATCH_SIZE, k+PATCH_SIZE))
            patch_array = np.array(patch)
            image_patches.append(patch_array)

# Convert the list of image patches to a numpy array
image_patches = np.array(image_patches)

# Reshape the image patches into sequences of vectors
seq_length = image_patches.shape[0] // labels_df.shape[0]
image_seqs = image_patches.reshape((-1, seq_length, PATCH_SIZE*PATCH_SIZE*3))

# Extract the labels and convert to one-hot encoding
labels = labels_df["gender"].values
encoder = LabelEncoder()
encoder.fit(labels)
labels = encoder.transform(labels)
labels = to_categorical(labels, num_classes=2)

# Split the data into training and testing sets
train_images, test_images, train_labels, test_labels = train_test_split(image_seqs, labels, test_size=0.2, random_state=42)

# Define the LSTM model
model = Sequential()
model.add(LSTM(128, input_shape=(seq_length, PATCH_SIZE*PATCH_SIZE*3)))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

# Train the model
model.fit(train_images, train_labels, epochs=10, batch_size=32, validation_data=(test_images, test_labels))
test_loss, test_acc = model.evaluate(test_images, test_labels)
print("Test loss:", test_loss)
print("Test accuracy:", test_acc)


Epoch 1/10
277/277 [==============================] - 178s 620ms/step - loss: 0.6664 - accuracy: 0.6291 - val_loss: 0.6537 - val_accuracy: 0.6462
Epoch 2/10
277/277 [==============================] - 166s 600ms/step - loss: 0.6575 - accuracy: 0.6407 - val_loss: 0.6532 - val_accuracy: 0.6462
Epoch 3/10
277/277 [==============================] - 164s 593ms/step - loss: 0.6566 - accuracy: 0.6407 - val_loss: 0.6509 - val_accuracy: 0.6462
Epoch 4/10
277/277 [==============================] - 149s 539ms/step - loss: 0.6554 - accuracy: 0.6407 - val_loss: 0.6498 - val_accuracy: 0.6462
Epoch 5/10
277/277 [==============================] - 148s 534ms/step - loss: 0.6558 - accuracy: 0.6407 - val_loss: 0.6522 - val_accuracy: 0.6462
Epoch 6/10
277/277 [==============================] - 152s 549ms/step - loss: 0.6552 - accuracy: 0.6407 - val_loss: 0.6507 - val_accuracy: 0.6462
Epoch 7/10
277/277 [==============================] - 150s 543ms/step - loss: 0.6553 - accuracy: 0.6407 - val_loss: 0.6512 -

# VGG16

In [1]:
import os
import pandas as pd
import cv2
import numpy as np
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

from PIL import Image
import os
import numpy as np

# Define the paths to the image and label files
IMG_PATH = "E:/mtech/secondsem/CV_Lab/Project/Hands/Hands" #whole images
LABELS_PATH = "E:/mtech/secondsem/CV_Lab/Project/HandInfo.csv"

# Load the labels
labels_df = pd.read_csv(LABELS_PATH)

# Define a list to store the images
images = []

# Load and preprocess each image
for i, row in labels_df.iterrows():
    img_path = os.path.join(IMG_PATH, row["imageName"])
    img = Image.open(img_path)
    img = img.resize((64, 64))
    img_array = np.array(img)
    images.append(img_array)

# Convert the list of images to a numpy array
images = np.array(images)

# Extract the labels and convert to one-hot encoding
labels = labels_df["gender"].values
encoder = LabelEncoder()
encoder.fit(labels)
labels = encoder.transform(labels)
labels = to_categorical(labels, num_classes=2)

# Split the data into training and testing sets
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)


# Define the model architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(train_images, train_labels, epochs=10, batch_size=32, validation_data=(test_images, test_labels))
test_loss, test_acc = model.evaluate(test_images, test_labels)
print("Test loss:", test_loss)
print("Test accuracy:", test_acc)

Epoch 1/10
277/277 [==============================] - 108s 381ms/step - loss: 2.5808 - accuracy: 0.6755 - val_loss: 0.7092 - val_accuracy: 0.5776
Epoch 2/10
277/277 [==============================] - 108s 391ms/step - loss: 0.4421 - accuracy: 0.8074 - val_loss: 0.3656 - val_accuracy: 0.8416
Epoch 3/10
277/277 [==============================] - 107s 387ms/step - loss: 0.2999 - accuracy: 0.8761 - val_loss: 0.3885 - val_accuracy: 0.8132
Epoch 4/10
277/277 [==============================] - 106s 382ms/step - loss: 0.2544 - accuracy: 0.8985 - val_loss: 0.2091 - val_accuracy: 0.9039
Epoch 5/10
277/277 [==============================] - 112s 404ms/step - loss: 0.2303 - accuracy: 0.9168 - val_loss: 0.1462 - val_accuracy: 0.9377
Epoch 6/10
277/277 [==============================] - 114s 412ms/step - loss: 0.1780 - accuracy: 0.9327 - val_loss: 0.1702 - val_accuracy: 0.9296
Epoch 7/10
277/277 [==============================] - 112s 405ms/step - loss: 0.1526 - accuracy: 0.9416 - val_loss: 0.1116 -

# Resnet50

In [3]:
import os
import pandas as pd
import cv2
import numpy as np
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from keras.utils import to_categorical

from PIL import Image
import os
import numpy as np

# Define the paths to the image and label files
IMG_PATH = "E:/mtech/secondsem/CV_Lab/Project/Hands/Hands" #whole images
LABELS_PATH = "E:/mtech/secondsem/CV_Lab/Project/HandInfo.csv"

# Load the labels
labels_df = pd.read_csv(LABELS_PATH)

# Define a list to store the images
images = []

# Load and preprocess each image
for i, row in labels_df.iterrows():
    img_path = os.path.join(IMG_PATH, row["imageName"])
    img = Image.open(img_path)
    img = img.resize((224, 224)) # ResNet50 model takes input size of (224,224)
    img_array = np.array(img)
    images.append(img_array)

# Convert the list of images to a numpy array
images = np.array(images)

# Extract the labels and convert to one-hot encoding
labels = labels_df["gender"].values
encoder = LabelEncoder()
encoder.fit(labels)
labels = encoder.transform(labels)
labels = to_categorical(labels, num_classes=2)

# Split the data into training and testing sets
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

# Load the pre-trained ResNet50 model
resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers in the pre-trained model
for layer in resnet.layers:
    layer.trainable = False

# Add new layers on top of the pre-trained model
x = Flatten()(resnet.output)
x = Dense(128, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)
model = Model(inputs=resnet.input, outputs=predictions)

# Compile the model
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(train_images, train_labels, epochs=10, batch_size=32, validation_data=(test_images, test_labels))
test_loss, test_acc = model.evaluate(test_images, test_labels)
print("Test loss:", test_loss)
print("Test accuracy:", test_acc)




Epoch 1/10
277/277 [==============================] - 3965s 14s/step - loss: 3.6577 - accuracy: 0.8692 - val_loss: 0.2220 - val_accuracy: 0.9725
Epoch 2/10
277/277 [==============================] - 3898s 14s/step - loss: 0.5698 - accuracy: 0.9498 - val_loss: 0.0825 - val_accuracy: 0.9851
Epoch 3/10
277/277 [==============================] - 3950s 14s/step - loss: 0.2453 - accuracy: 0.9690 - val_loss: 0.0658 - val_accuracy: 0.9883
Epoch 4/10
277/277 [==============================] - 3957s 14s/step - loss: 0.1232 - accuracy: 0.9772 - val_loss: 0.0345 - val_accuracy: 0.9919
Epoch 5/10
277/277 [==============================] - 3927s 14s/step - loss: 0.0627 - accuracy: 0.9857 - val_loss: 0.0760 - val_accuracy: 0.9707
Epoch 6/10
277/277 [==============================] - 3895s 14s/step - loss: 0.0511 - accuracy: 0.9897 - val_loss: 0.0342 - val_accuracy: 0.9896
Epoch 7/10
277/277 [==============================] - 3882s 14s/step - loss: 0.0309 - accuracy: 0.9928 - val_loss: 0.0126 - val_ac

# Transformer Model

In [2]:
import os
import pandas as pd
import cv2
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation, RandomTranslation, RandomZoom, Rescaling
import tensorflow as tf
from PIL import Image
import os
import numpy as np

# Define the paths to the image and label files
IMG_PATH = "E:/mtech/secondsem/CV_Lab/Project/Hands/Hands" #whole images
LABELS_PATH = "E:/mtech/secondsem/CV_Lab/Project/HandInfo.csv"

# Load the labels
labels_df = pd.read_csv(LABELS_PATH)

# Define a list to store the images
images = []

# Load and preprocess each image
for i, row in labels_df.iterrows():
    img_path = os.path.join(IMG_PATH, row["imageName"])
    img = Image.open(img_path)
    img = img.resize((224, 224))
    img_array = np.array(img)
    images.append(img_array)

# Convert the list of images to a numpy array
images = np.array(images)

# Extract the labels and convert to one-hot encoding
labels = labels_df["gender"].values
encoder = LabelEncoder()
encoder.fit(labels)
labels = encoder.transform(labels)
labels = to_categorical(labels, num_classes=2)

# Split the data into training and testing sets
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

# Define the input shape
input_shape = (224, 224, 3)

# Define the transformer model
def transformer_model(input_shape):
    inputs = Input(shape=input_shape)
    
    # Data augmentation
    x = RandomFlip()(inputs)
    x = RandomRotation(factor=0.2)(x)
    x = RandomTranslation(height_factor=0.1, width_factor=0.1)(x)
    x = RandomZoom(height_factor=0.2, width_factor=0.2)(x)
    x = Rescaling(scale=1./255)(x)
    
    # Base model
    base_model = EfficientNetB0(include_top=False, weights="imagenet", input_tensor=x)
    x = base_model.output
    x = Flatten()(x)
    x = Dropout(0.5)(x)
    outputs = Dense(2, activation='softmax')(x)
    
    model = tf.keras.Model(inputs, outputs)
    
    return model

# Create an instance of the transformer model
model = transformer_model(input_shape)

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.0001),
              metrics=['accuracy'])

# Train the model
model.fit(train_images, train_labels, epochs=10, batch_size=32, validation_data=(test_images, test_labels))
test_loss, test_acc = model.evaluate(test_images, test_labels)
print("Test loss:", test_loss)
print("Test accuracy:", test_acc)


Epoch 1/10


277/277 [==============================] - 3093s 11s/step - loss: 0.8356 - accuracy: 0.7869 - val_loss: 0.9180 - val_accuracy: 0.6462
Epoch 2/10
277/277 [==============================] - 2804s 10s/step - loss: 0.9961 - accuracy: 0.7843 - val_loss: 73.9547 - val_accuracy: 0.6462
Epoch 3/10
277/277 [==============================] - 2837s 10s/step - loss: 1.1666 - accuracy: 0.7778 - val_loss: 4.9817 - val_accuracy: 0.6462
Epoch 4/10
277/277 [==============================] - 2842s 10s/step - loss: 0.8602 - accuracy: 0.8010 - val_loss: 9.4120 - val_accuracy: 0.6462
Epoch 5/10
277/277 [==============================] - 2838s 10s/step - loss: 0.5315 - accuracy: 0.8262 - val_loss: 66.0731 - val_accuracy: 0.6462
Epoch 6/10
277/277 [==============================] - 2855s 10s/step - loss: 0.6538 - accuracy: 0.8324 - val_loss: 3.1423 - val_accuracy: 0.3547
Epoch 7/10
277/277 [==============================] - 2897s 10s/step - loss: 1.0278 - accuracy: 0.7669 - val_loss: 3.9196 - val_accuracy: 0